# Preprocessing and clustering

## Loading import

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import anndata as ad
import gzip
import shutil
from scipy import sparse
import os
from anndata.experimental.multi_files import AnnCollection
from matplotlib.pyplot import rc_context
# import seaborn as sns
# import matplotlib.pyplot as plt

# %matplotlib inline
import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rc('font', size=14)
plt.rcParams['pdf.fonttype'] = 42

In [2]:
sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=120, frameon=False, figsize=(3, 3), facecolor='white')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.5 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.1 statsmodels==0.13.2 python-igraph==0.9.10 louvain==0.7.1 pynndescent==0.5.4


In [3]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

## Input data

In [4]:
# !mkdir write
# !mkdir tables

In [5]:
wdir = "/Users/xuewei/ZxProjects/CRC/2023_GSE231559/01_preprocessing/01_clustering_scrublet_231117/"
os.chdir( wdir )

### adata_Colon_Normal_1

In [45]:
# input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C1N/'
# counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()
# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
# out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

# print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
# print('Number of genes in gene list: {}'.format(len(genes)))        

In [9]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C1N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)
# scrub.plot_histogram();

# # 4. Runding UMAP
# print('Running UMAP...')
# scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))

# # # Uncomment to run tSNE - slow
# # print('Running tSNE...')
# # scrub.set_embedding('tSNE', scr.get_tsne(scrub.manifold_obs_, angle=0.9))

# # # Uncomment to run force layout - slow
# # print('Running ForceAtlas2...')
# # scrub.set_embedding('FA', scr.get_force_layout(scrub.manifold_obs_, n_neighbors=5. n_iter=1000))
    
# print('Done.')

# scrub.plot_embedding('UMAP', order_points=True);

# # scrub.plot_embedding('tSNE', order_points=True);
# # scrub.plot_embedding('FA', order_points=True);


# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

Counts matrix shape: 6132 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.61
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.3%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 6.5%
Elapsed time: 4.9 seconds


In [10]:
adata_Colon_Normal_1 = sc.read_10x_mtx(
    '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C1N/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=False)                              # write a cache file for faster subsequent reading

adata_Colon_Normal_1.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`
adata_Colon_Normal_1
# adata.var["gene_ids"]
# adata.var["feature_types"]
# adata.to_df()
# type(adata.var["gene_ids"])

# adata_NAG1.write(results_file)

--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


AnnData object with n_obs × n_vars = 6132 × 33538
    var: 'gene_ids', 'feature_types'

In [11]:
doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

In [13]:
adata_Colon_Normal_1.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Colon_Normal_1.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Colon_Normal_1 = adata_Colon_Normal_1[adata_Colon_Normal_1.obs['predicted_doublets'] == False]
adata_Colon_Normal_1

View of AnnData object with n_obs × n_vars = 6131 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Colon_Tumor_1

In [14]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C1T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Colon_Tumor_1 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C1T/', var_names='gene_symbols', cache=False)
adata_Colon_Tumor_1.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Colon_Tumor_1.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Colon_Tumor_1.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Colon_Tumor_1 = adata_Colon_Tumor_1[adata_Colon_Tumor_1.obs['predicted_doublets'] == False]
adata_Colon_Tumor_1

Counts matrix shape: 6257 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.62
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.4%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 3.8%
Elapsed time: 5.4 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 6256 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Colon_Normal_2 and adata_Colon_Tumor_2

In [16]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C2N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Colon_Normal_2 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C2N/', var_names='gene_symbols', cache=False)
adata_Colon_Normal_2.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Colon_Normal_2.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Colon_Normal_2.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Colon_Normal_2 = adata_Colon_Normal_2[adata_Colon_Normal_2.obs['predicted_doublets'] == False]
adata_Colon_Normal_2

Counts matrix shape: 8132 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.64
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.3%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 8.7%
Elapsed time: 5.5 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 8130 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

In [17]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C2T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Colon_Tumor_2 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C2T/', var_names='gene_symbols', cache=False)
adata_Colon_Tumor_2.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Colon_Tumor_2.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Colon_Tumor_2.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Colon_Tumor_2 = adata_Colon_Tumor_2[adata_Colon_Tumor_2.obs['predicted_doublets'] == False]
adata_Colon_Tumor_2

Counts matrix shape: 4799 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.57
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 1.9%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 0.0%
Elapsed time: 3.6 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 4799 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Colon_Normal_3 and adata_Colon_Tumor_3

In [18]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C3N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Colon_Normal_3 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C3N/', var_names='gene_symbols', cache=False)
adata_Colon_Normal_3.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Colon_Normal_3.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Colon_Normal_3.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Colon_Normal_3 = adata_Colon_Normal_3[adata_Colon_Normal_3.obs['predicted_doublets'] == False]
adata_Colon_Normal_3

Counts matrix shape: 5942 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.61
Detected doublet rate = 0.2%
Estimated detectable doublet fraction = 2.5%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 8.1%
Elapsed time: 4.5 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 5930 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

In [19]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C3T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Colon_Tumor_3 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C3T/', var_names='gene_symbols', cache=False)
adata_Colon_Tumor_3.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Colon_Tumor_3.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Colon_Tumor_3.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Colon_Tumor_3 = adata_Colon_Tumor_3[adata_Colon_Tumor_3.obs['predicted_doublets'] == False]
adata_Colon_Tumor_3

Counts matrix shape: 8951 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.66
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 0.8%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 8.4%
Elapsed time: 7.6 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 8945 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Colon_Tumor_4, adata_Colon_Tumor_5 and adata_Colon_Tumor_6

In [20]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C4T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Colon_Tumor_4 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C4T/', var_names='gene_symbols', cache=False)
adata_Colon_Tumor_4.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Colon_Tumor_4.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Colon_Tumor_4.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Colon_Tumor_4 = adata_Colon_Tumor_4[adata_Colon_Tumor_4.obs['predicted_doublets'] == False]
adata_Colon_Tumor_4

Counts matrix shape: 3361 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.51
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 9.0%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 1.3%
Elapsed time: 2.9 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 3357 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

In [21]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C5T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Colon_Tumor_5 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C5T/', var_names='gene_symbols', cache=False)
adata_Colon_Tumor_5.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Colon_Tumor_5.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Colon_Tumor_5.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Colon_Tumor_5 = adata_Colon_Tumor_5[adata_Colon_Tumor_5.obs['predicted_doublets'] == False]
adata_Colon_Tumor_5

Counts matrix shape: 4889 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.58
Detected doublet rate = 0.2%
Estimated detectable doublet fraction = 2.0%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 8.0%
Elapsed time: 4.2 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 4881 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

In [22]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C6T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Colon_Tumor_6 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/C6T/', var_names='gene_symbols', cache=False)
adata_Colon_Tumor_6.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Colon_Tumor_6.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Colon_Tumor_6.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Colon_Tumor_6 = adata_Colon_Tumor_6[adata_Colon_Tumor_6.obs['predicted_doublets'] == False]
adata_Colon_Tumor_6

Counts matrix shape: 4428 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.57
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 2.9%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 4.0%
Elapsed time: 3.2 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 4423 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Normal_1 and adata_Liver_Tumor_1

In [23]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L1N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Normal_1 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L1N/', var_names='gene_symbols', cache=False)
adata_Liver_Normal_1.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Colon_Tumor_6.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Colon_Tumor_6.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Colon_Tumor_6 = adata_Colon_Tumor_6[adata_Colon_Tumor_6.obs['predicted_doublets'] == False]
adata_Colon_Tumor_6

Counts matrix shape: 3570 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.54
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 4.8%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 2.3%
Elapsed time: 2.3 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


/var/folders/nt/95vmy3614zd_8x3sv2ktwn7c0000gp/T/ipykernel_38254/409797821.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_Colon_Tumor_6.obs['doublet_scores'] = doublet_data['doublet_scores']


View of AnnData object with n_obs × n_vars = 6 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

In [24]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L1T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Tumor_1 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L1T/', var_names='gene_symbols', cache=False)
adata_Liver_Tumor_1.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Tumor_1.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Tumor_1.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Tumor_1 = adata_Liver_Tumor_1[adata_Liver_Tumor_1.obs['predicted_doublets'] == False]
adata_Liver_Tumor_1

Counts matrix shape: 3275 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.54
Detected doublet rate = 0.2%
Estimated detectable doublet fraction = 4.8%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 3.8%
Elapsed time: 2.6 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 3269 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Normal_2

In [25]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L2N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Normal_2 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L2N/', var_names='gene_symbols', cache=False)
adata_Liver_Normal_2.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Normal_2.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Normal_2.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Normal_2 = adata_Liver_Normal_2[adata_Liver_Normal_2.obs['predicted_doublets'] == False]
adata_Liver_Normal_2

Counts matrix shape: 3330 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.32
Detected doublet rate = 1.4%
Estimated detectable doublet fraction = 26.4%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 5.3%
Elapsed time: 2.2 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 3283 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Normal_3

In [26]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L3N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Normal_3 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L3N/', var_names='gene_symbols', cache=False)
adata_Liver_Normal_3.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Normal_3.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Normal_3.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Normal_3 = adata_Liver_Normal_3[adata_Liver_Normal_3.obs['predicted_doublets'] == False]
adata_Liver_Normal_3

Counts matrix shape: 4880 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.38
Detected doublet rate = 1.2%
Estimated detectable doublet fraction = 22.7%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 5.1%
Elapsed time: 3.3 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 4823 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Normal_4 and adata_Liver_Tumor_4

In [27]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L4N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Normal_4 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L4N/', var_names='gene_symbols', cache=False)
adata_Liver_Normal_4.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Normal_4.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Normal_4.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Normal_4 = adata_Liver_Normal_4[adata_Liver_Normal_4.obs['predicted_doublets'] == False]
adata_Liver_Normal_4

Counts matrix shape: 3009 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.52
Detected doublet rate = 0.5%
Estimated detectable doublet fraction = 8.6%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 6.2%
Elapsed time: 2.2 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 2993 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

In [28]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L4T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Tumor_4 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L4T/', var_names='gene_symbols', cache=False)
adata_Liver_Tumor_4.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Tumor_4.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Tumor_4.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Tumor_4 = adata_Liver_Tumor_4[adata_Liver_Tumor_4.obs['predicted_doublets'] == False]
adata_Liver_Tumor_4

Counts matrix shape: 3999 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.36
Detected doublet rate = 0.6%
Estimated detectable doublet fraction = 16.8%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 3.7%
Elapsed time: 3.5 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 3974 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Normal_5

In [30]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L5N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Normal_5 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L5N/', var_names='gene_symbols', cache=False)
adata_Liver_Normal_5.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Normal_5.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Normal_5.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Normal_5 = adata_Liver_Normal_5[adata_Liver_Normal_5.obs['predicted_doublets'] == False]
adata_Liver_Normal_5

Counts matrix shape: 3375 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.34
Detected doublet rate = 0.8%
Estimated detectable doublet fraction = 16.1%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 5.2%
Elapsed time: 2.2 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 3347 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Tumor_6

In [29]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L6T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Tumor_6 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L6T/', var_names='gene_symbols', cache=False)
adata_Liver_Tumor_6.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Tumor_6.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Tumor_6.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Tumor_6 = adata_Liver_Tumor_6[adata_Liver_Tumor_6.obs['predicted_doublets'] == False]
adata_Liver_Tumor_6

Counts matrix shape: 6082 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.35
Detected doublet rate = 1.2%
Estimated detectable doublet fraction = 21.0%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 5.6%
Elapsed time: 4.3 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 6011 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Normal_7

In [31]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L7N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Normal_7 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L7N/', var_names='gene_symbols', cache=False)
adata_Liver_Normal_7.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Normal_7.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Normal_7.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Normal_7 = adata_Liver_Normal_7[adata_Liver_Normal_7.obs['predicted_doublets'] == False]
adata_Liver_Normal_7

Counts matrix shape: 1312 rows, 33538 columns
Number of genes in gene list: 33538
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.42
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 4.6%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 0.0%
Elapsed time: 0.7 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 1312 × 33538
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Tumor_8_1 and adata_Liver_Tumor_8_2

In [32]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L8T1/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Tumor_8_1 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L8T1/', var_names='gene_symbols', cache=False)
adata_Liver_Tumor_8_1.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Tumor_8_1.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Tumor_8_1.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Tumor_8_1 = adata_Liver_Tumor_8_1[adata_Liver_Tumor_8_1.obs['predicted_doublets'] == False]
adata_Liver_Tumor_8_1

Counts matrix shape: 3123 rows, 36601 columns
Number of genes in gene list: 36601
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.52
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 2.4%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 2.7%
Elapsed time: 2.1 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 3121 × 36601
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

In [33]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L8T2/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Tumor_8_2 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L8T2/', var_names='gene_symbols', cache=False)
adata_Liver_Tumor_8_2.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Tumor_8_2.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Tumor_8_2.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Tumor_8_2 = adata_Liver_Tumor_8_2[adata_Liver_Tumor_8_2.obs['predicted_doublets'] == False]
adata_Liver_Tumor_8_2

Counts matrix shape: 1646 rows, 36601 columns
Number of genes in gene list: 36601
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.42
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 7.9%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 1.5%
Elapsed time: 1.0 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 1644 × 36601
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Normal_9 and adata_Liver_Tumor_9

In [34]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L9N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Normal_9 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L9N/', var_names='gene_symbols', cache=False)
adata_Liver_Normal_9.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Normal_9.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Normal_9.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Normal_9 = adata_Liver_Normal_9[adata_Liver_Normal_9.obs['predicted_doublets'] == False]
adata_Liver_Normal_9

Counts matrix shape: 5218 rows, 36601 columns
Number of genes in gene list: 36601
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.30
Detected doublet rate = 1.0%
Estimated detectable doublet fraction = 20.6%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 4.8%
Elapsed time: 4.0 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 5166 × 36601
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

In [35]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L9T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Tumor_9 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L9T/', var_names='gene_symbols', cache=False)
adata_Liver_Tumor_9.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Tumor_9.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Tumor_9.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Tumor_9 = adata_Liver_Tumor_9[adata_Liver_Tumor_9.obs['predicted_doublets'] == False]
adata_Liver_Tumor_9

Counts matrix shape: 10009 rows, 36601 columns
Number of genes in gene list: 36601
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.59
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.2%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 0.0%
Elapsed time: 9.0 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 10009 × 36601
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Tumor_10

In [36]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L10T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Tumor_10 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L10T/', var_names='gene_symbols', cache=False)
adata_Liver_Tumor_10.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Tumor_10.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Tumor_10.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Tumor_10 = adata_Liver_Tumor_10[adata_Liver_Tumor_10.obs['predicted_doublets'] == False]
adata_Liver_Tumor_10

Counts matrix shape: 8378 rows, 36601 columns
Number of genes in gene list: 36601
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.65
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 1.2%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 1.9%
Elapsed time: 6.0 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 8376 × 36601
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Normal_11 and adata_Liver_Tumor_11

In [37]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L11N/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Normal_11 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L11N/', var_names='gene_symbols', cache=False)
adata_Liver_Normal_11.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Normal_11.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Normal_11.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Normal_11 = adata_Liver_Normal_11[adata_Liver_Normal_11.obs['predicted_doublets'] == False]
adata_Liver_Normal_11

Counts matrix shape: 274 rows, 36601 columns
Number of genes in gene list: 36601
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.25
Detected doublet rate = 7.3%
Estimated detectable doublet fraction = 5.8%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 125.0%
Elapsed time: 0.1 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 254 × 36601
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

In [38]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L11T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Tumor_11 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L11T/', var_names='gene_symbols', cache=False)
adata_Liver_Tumor_11.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Tumor_11.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Tumor_11.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Tumor_11 = adata_Liver_Tumor_11[adata_Liver_Tumor_11.obs['predicted_doublets'] == False]
adata_Liver_Tumor_11

Counts matrix shape: 9476 rows, 36601 columns
Number of genes in gene list: 36601
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.58
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.2%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 8.5%
Elapsed time: 7.3 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 9474 × 36601
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### adata_Liver_Tumor_12

In [39]:
# 1. input_dir
input_dir = '/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L12T/'

# 2. input
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx.gz').T.tocsc()

# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))
gz_file = input_dir + '/features.tsv.gz'
decompressed_file = input_dir + '/features.tsv'
with gzip.open(gz_file, 'rb') as f_in:
    with open(decompressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
genes = np.array(scr.load_genes(decompressed_file, delimiter='\t', column=1))
os.remove(decompressed_file)

out_df = pd.read_csv(input_dir + '/barcodes.tsv.gz', header = None, index_col=None, names=['barcode'])

print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
print('Number of genes in gene list: {}'.format(len(genes)))

# 3. Scrublet
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

# 5. save doublet.txt
scrub.detected_doublet_rate_

out_df['doublet_scores'] = doublet_scores
out_df['predicted_doublets'] = predicted_doublets
out_df.to_csv(input_dir + '/doublet.txt', index=False,header=True)
# out_df.head()

# 6. input data
adata_Liver_Tumor_12 = sc.read_10x_mtx('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/00_backup/GSE231559_RAW/L12T/', var_names='gene_symbols', cache=False)
adata_Liver_Tumor_12.var_names_make_unique()

doublet_data = pd.read_csv(input_dir + 'doublet.txt', header=0, index_col=0)

adata_Liver_Tumor_12.obs['doublet_scores'] = doublet_data['doublet_scores']
adata_Liver_Tumor_12.obs['predicted_doublets'] = doublet_data['predicted_doublets']
adata_Liver_Tumor_12 = adata_Liver_Tumor_12[adata_Liver_Tumor_12.obs['predicted_doublets'] == False]
adata_Liver_Tumor_12

Counts matrix shape: 10119 rows, 36601 columns
Number of genes in gene list: 36601
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.42
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.0%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 57.1%
Elapsed time: 8.3 seconds
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.


View of AnnData object with n_obs × n_vars = 10117 × 36601
    obs: 'doublet_scores', 'predicted_doublets'
    var: 'gene_ids', 'feature_types'

### Label with sample id

In [40]:
adata_Colon_Normal_1.obs["batch"] = "Col_N_1"
adata_Colon_Tumor_1.obs["batch"] = "Col_T_1"

adata_Colon_Normal_2.obs["batch"] = "Col_N_2"
adata_Colon_Tumor_2.obs["batch"] = "Col_T_2"

adata_Colon_Normal_3.obs["batch"] = "Col_N_3"
adata_Colon_Tumor_3.obs["batch"] = "Col_T_3"

adata_Colon_Tumor_4.obs["batch"] = "Col_T_4"
adata_Colon_Tumor_5.obs["batch"] = "Col_T_5"
adata_Colon_Tumor_6.obs["batch"] = "Col_T_6"

adata_Liver_Normal_1.obs["batch"] = "Liv_N_1"
adata_Liver_Tumor_1.obs["batch"] = "Liv_T_1"

adata_Liver_Normal_2.obs["batch"] = "Liv_N_2"

adata_Liver_Normal_3.obs["batch"] = "Liv_N_3"

adata_Liver_Normal_4.obs["batch"] = "Liv_N_4"
adata_Liver_Tumor_4.obs["batch"] = "Liv_T_4"

adata_Liver_Normal_5.obs["batch"] = "Liv_N_5"

adata_Liver_Tumor_6.obs["batch"] = "Liv_T_6"

adata_Liver_Normal_7.obs["batch"] = "Liv_N_7"

adata_Liver_Tumor_8_1.obs["batch"] = "Liv_T_8_1"
adata_Liver_Tumor_8_2.obs["batch"] = "Liv_T_8_2"

adata_Liver_Normal_9.obs["batch"] = "Liv_N_9"
adata_Liver_Tumor_9.obs["batch"] = "Liv_T_9"

adata_Liver_Tumor_10.obs["batch"] = "Liv_T_10"

adata_Liver_Normal_11.obs["batch"] = "Liv_N_11"
adata_Liver_Tumor_11.obs["batch"] = "Liv_T_11"

adata_Liver_Tumor_12.obs["batch"] = "Liv_T_12"

/var/folders/nt/95vmy3614zd_8x3sv2ktwn7c0000gp/T/ipykernel_38254/297197394.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_Colon_Normal_1.obs["batch"] = "Col_N_1"
/var/folders/nt/95vmy3614zd_8x3sv2ktwn7c0000gp/T/ipykernel_38254/297197394.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_Colon_Tumor_1.obs["batch"] = "Col_T_1"
/var/folders/nt/95vmy3614zd_8x3sv2ktwn7c0000gp/T/ipykernel_38254/297197394.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_Colon_Normal_2.obs["batch"] = "Col_N_2"
/var/folders/nt/95vmy3614zd_8x3sv2ktwn7c0000gp/T/ipykernel_38254/297197394.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_Colon_Tumor_2.obs["batch"] = "Col_T_2"
/var/folders/nt/95vmy3614zd_8x3sv2ktwn7c0000gp/T/ipykernel_38254/297197394.py:7: I

In [41]:
print("adata_Colon_Normal_1:", adata_Colon_Normal_1.shape,
    "\nadata_Colon_Tumor_1:", adata_Colon_Tumor_1.shape, 
    "\nadata_Colon_Normal_2:", adata_Colon_Normal_2.shape, 
    "\nadata_Colon_Tumor_2:", adata_Colon_Tumor_2.shape, 
    "\nadata_Colon_Normal_3:", adata_Colon_Normal_3.shape, 
    "\nadata_Colon_Tumor_3:", adata_Colon_Tumor_3.shape, 
    "\nadata_Colon_Tumor_4:", adata_Colon_Tumor_4.shape, 
    "\nadata_Colon_Tumor_5:", adata_Colon_Tumor_5.shape, 
    "\nadata_Colon_Tumor_6:", adata_Colon_Tumor_6.shape, 
    "\nadata_Liver_Normal_1:", adata_Liver_Normal_1.shape, 
    "\nadata_Liver_Tumor_1:", adata_Liver_Tumor_1.shape, 
    "\nadata_Liver_Normal_2:", adata_Liver_Normal_2.shape, 
    "\nadata_Liver_Normal_3:", adata_Liver_Normal_3.shape, 
    "\nadata_Liver_Normal_4:", adata_Liver_Normal_4.shape, 
    "\nadata_Liver_Tumor_4:", adata_Liver_Tumor_4.shape, 
    "\nadata_Liver_Normal_5:", adata_Liver_Normal_5.shape, 
    "\nadata_Liver_Tumor_6:", adata_Liver_Tumor_6.shape, 
    "\nadata_Liver_Normal_7:", adata_Liver_Normal_7.shape, 
    "\nadata_Liver_Tumor_8_1:", adata_Liver_Tumor_8_1.shape, 
    "\nadata_Liver_Tumor_8_2:", adata_Liver_Tumor_8_2.shape, 
    "\nadata_Liver_Normal_9:", adata_Liver_Normal_9.shape, 
    "\nadata_Liver_Tumor_9:", adata_Liver_Tumor_9.shape, 
    "\nadata_Liver_Tumor_10:", adata_Liver_Tumor_10.shape, 
    "\nadata_Liver_Normal_11:", adata_Liver_Normal_11.shape, 
    "\nadata_Liver_Tumor_11:", adata_Liver_Tumor_11.shape, 
    "\nadata_Liver_Tumor_12:", adata_Liver_Tumor_12.shape, 
    
    )

adata_Colon_Normal_1: (6131, 33538) 
adata_Colon_Tumor_1: (6256, 33538) 
adata_Colon_Normal_2: (8130, 33538) 
adata_Colon_Tumor_2: (4799, 33538) 
adata_Colon_Normal_3: (5930, 33538) 
adata_Colon_Tumor_3: (8945, 33538) 
adata_Colon_Tumor_4: (3357, 33538) 
adata_Colon_Tumor_5: (4881, 33538) 
adata_Colon_Tumor_6: (6, 33538) 
adata_Liver_Normal_1: (3570, 33538) 
adata_Liver_Tumor_1: (3269, 33538) 
adata_Liver_Normal_2: (3283, 33538) 
adata_Liver_Normal_3: (4823, 33538) 
adata_Liver_Normal_4: (2993, 33538) 
adata_Liver_Tumor_4: (3974, 33538) 
adata_Liver_Normal_5: (3347, 33538) 
adata_Liver_Tumor_6: (6011, 33538) 
adata_Liver_Normal_7: (1312, 33538) 
adata_Liver_Tumor_8_1: (3121, 36601) 
adata_Liver_Tumor_8_2: (1644, 36601) 
adata_Liver_Normal_9: (5166, 36601) 
adata_Liver_Tumor_9: (10009, 36601) 
adata_Liver_Tumor_10: (8376, 36601) 
adata_Liver_Normal_11: (254, 36601) 
adata_Liver_Tumor_11: (9474, 36601) 
adata_Liver_Tumor_12: (10117, 36601)


In [42]:
adList = [adata_Colon_Normal_1, adata_Colon_Tumor_1, adata_Colon_Normal_2, adata_Colon_Normal_2, adata_Colon_Normal_3, adata_Colon_Tumor_3,
          adata_Colon_Tumor_4, adata_Colon_Tumor_5, adata_Colon_Tumor_5, 
          adata_Liver_Normal_1, adata_Liver_Tumor_1, adata_Liver_Normal_2, adata_Liver_Normal_3, adata_Liver_Normal_4, adata_Liver_Tumor_4,
          adata_Liver_Normal_5, adata_Liver_Tumor_6, adata_Liver_Normal_7, adata_Liver_Tumor_8_1, adata_Liver_Tumor_8_2, adata_Liver_Normal_9, 
          adata_Liver_Tumor_9, adata_Liver_Tumor_10, adata_Liver_Normal_11, adata_Liver_Tumor_11, adata_Liver_Tumor_12]
ad_all = ad.concat(adList, join='outer')
ad_all.var_names_make_unique()
ad_all.obs_names_make_unique()
sc.pp.calculate_qc_metrics(ad_all, percent_top=None,log1p=False,inplace=True)

/Users/xuewei/ZeroData/softwares/miniconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [26]:
# ad_all2 = sc.AnnData.concatenate(adata_NT1, adata_PT1, adata_PT2, adata_PT3, adata_LN1, adata_LN2, adata_Li1, adata_Li2, adata_OV1, adata_PM1, join='outer')
# ad_all2.var_names_make_unique()
# ad_all2.obs_names_make_unique()
# sc.pp.calculate_qc_metrics(ad_all2, percent_top=None,log1p=False,inplace=True)
# ad_all2.obs['batch2'] = ad_all2.obs['batch'].replace(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], ['NT1', 'PT1', "PT2", "PT3", "LN1", "LN2", "Li1", "Li2", "OV1", "PM1"])

In [43]:
adata = ad_all
adata

AnnData object with n_obs × n_vars = 137384 × 38224
    obs: 'doublet_scores', 'predicted_doublets', 'batch', 'n_genes_by_counts', 'total_counts'
    var: 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [47]:
adata.obs

,doublet_scores,predicted_doublets,batch,n_genes_by_counts,total_counts
AAACCCAAGAGAGCCT-1,0.059294,False,Col_N_1,689,3664.0
AAACCCAAGGGTATAT-1,0.034660,False,Col_N_1,66,779.0
AAACCCAAGTCCCGAC-1,0.073892,False,Col_N_1,1811,11293.0
AAACCCACATTAAGCC-1,0.038224,False,Col_N_1,998,2686.0
AAACCCAGTGTTCCAA-1,0.055276,False,Col_N_1,192,741.0
...,...,...,...,...,...
TTTGTTGAGAGGTTTA-1,0.110772,False,Liv_T_12,953,2525.0
TTTGTTGAGCTACGTT-1,0.016799,False,Liv_T_12,305,583.0
TTTGTTGCACACCTTC-1,0.016799,False,Liv_T_12,224,588.0
TTTGTTGCATATCTGG-1,0.017282,False,Liv_T_12,300,840.0


In [48]:
adata_backup = adata.copy()

In [49]:
adata.obs.drop(['doublet_scores', 'predicted_doublets'], axis=1, inplace=True)

In [51]:
adata.obs

,batch,n_genes_by_counts,total_counts
AAACCCAAGAGAGCCT-1,Col_N_1,689,3664.0
AAACCCAAGGGTATAT-1,Col_N_1,66,779.0
AAACCCAAGTCCCGAC-1,Col_N_1,1811,11293.0
AAACCCACATTAAGCC-1,Col_N_1,998,2686.0
AAACCCAGTGTTCCAA-1,Col_N_1,192,741.0
...,...,...,...
TTTGTTGAGAGGTTTA-1,Liv_T_12,953,2525.0
TTTGTTGAGCTACGTT-1,Liv_T_12,305,583.0
TTTGTTGCACACCTTC-1,Liv_T_12,224,588.0
TTTGTTGCATATCTGG-1,Liv_T_12,300,840.0


In [52]:
adata.write_h5ad('/Users/xuewei/ZxProjects/CRC/2023_GSE231559/01_preprocessing/write/CRC_00_merge_20231117.h5ad')